In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import json

import re
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer

from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Import Data

In [ ]:
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
sample_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")

In [ ]:
sample_df.head()

In [ ]:
test_path = "../input/coleridgeinitiative-show-us-the-data/test"
all_test_path = [os.path.join(test_path,s) + ".json" for s in sample_df["Id"]]

In [ ]:
alltexts = []

for json_path in tqdm(all_test_path):

    with open(json_path, 'r') as f:
            json_decode = json.load(f)
    jsontest = pd.DataFrame(json_decode)

    texts = ""

    for a in jsontest.values:
        texts += a[0] + " " + a[1] + " "
        
    alltexts.append(texts)

In [ ]:
sample_df["text"] = alltexts

In [ ]:
train_path = "../input/coleridgeinitiative-show-us-the-data/train"
all_train_path = [os.path.join(train_path,s) + ".json" for s in train_df["Id"]]

In [ ]:
alltexts = []

for json_path in tqdm(all_train_path):

    with open(json_path, 'r') as f:
            json_decode = json.load(f)
    jsontest = pd.DataFrame(json_decode)

    texts = ""

    for a in jsontest.values:
        texts += a[0] + " " + a[1] + " "
        
    alltexts.append(texts)

In [ ]:
train_df["text"] = alltexts

# Figure Train Text

In [ ]:
cleaned = list(train_df['cleaned_label'])

cleans = []

for i in cleaned:
    i = list(i.split(" "))
    for j in i:
        cleans.append(j.replace(" ", ""))

cln = Counter(cleans)

In [ ]:
clns = list(set(cln))
plt.rcParams['figure.figsize'] = (15,15)
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = 'white').generate(str(clns))

plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Cleaned Label Distribution', size = 23, color = 'darkblue')
plt.show()



In [ ]:
from collections import Counter

cleaned = list(train_df['dataset_label'])

cleans = []

for i in cleaned:
    i = list(i.split(" "))
    for j in i:
        cleans.append(j.replace(" ", ""))

cln = Counter(cleans)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

clns = list(set(cln))
plt.rcParams['figure.figsize'] = (15,15)
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = 'white').generate(str(clns))

plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Dataset Label Distribution', size = 23, color = 'darkblue')
plt.show()



In [ ]:
from collections import Counter

cleaned = list(train_df['dataset_title'])

cleans = []

for i in cleaned:
    i = list(i.split(" "))
    for j in i:
        cleans.append(j.replace(" ", ""))

cln = Counter(cleans)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

clns = list(set(cln))
plt.rcParams['figure.figsize'] = (15,15)
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = 'white').generate(str(clns))

plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Dataset Title Distribution', size = 23, color = 'darkblue')
plt.show()



In [ ]:
from collections import Counter

cleaned = list(train_df['pub_title'])

cleans = []

for i in cleaned:
    i = list(i.split(" "))
    for j in i:
        cleans.append(j.replace(" ", ""))

cln = Counter(cleans)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

clns = list(set(cln))
plt.rcParams['figure.figsize'] = (15,15)
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = 'white').generate(str(clns))

plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Pub Title Distribution', size = 23, color = 'darkblue')
plt.show()



# Combine Datasets

In [ ]:
porter = PorterStemmer()
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def clean_text(text):
    text= re.sub('r[^\w\s]',' ',text)
    #text= re.sub('[^a-zA-z0-9\s]','',text)
    text=lower_case(text)
    text=remove_stopwords(text)
    text=stemSentence(text)    
    return text

In [ ]:
temp_1 = [x.lower() for x in train_df['dataset_label'].unique()]
temp_2 = [x.lower() for x in train_df['dataset_title'].unique()]
temp_3 = [x.lower() for x in train_df['cleaned_label'].unique()]
temp_4 = [x.lower() for x in train_df['pub_title'].unique()]
temp_5 = [x.lower() for x in train_df['text'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3 + temp_4  + temp_5)

In [ ]:
id_list = []
lables_list = []
for index, row in tqdm(sample_df.iterrows()):
    sample_text = row['text']
    row_id = row['Id']

    temp_df = train_df[train_df['text'] == clean_text(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    for known_label in existing_labels:
        if known_label in sample_text.lower():
            cleaned_labels.append(clean_text(known_label))
    cleaned_labels = [clean_text(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    lables_list.append(' | '.join(cleaned_labels))
    id_list.append(row_id)

> Making submission

In [ ]:
sample_df["PredictionString"] = lables_list

In [ ]:
del sample_df['text']

In [ ]:
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv', index = False)